# Demo Code with ML

(10/27/2022) 1st version completed 


## Import

In [1]:
import cv2
import os
import math
import numpy as np
import pandas as pd
import pickle
import mediapipe as mp

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Sklearn
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix # Helps present results as a confusion-matrix

# Import of keras model and hidden layers for convolutional network
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten

# Model
from lightgbm import LGBMClassifier
from lightgbm import plot_importance 
from xgboost import XGBClassifier
from xgboost import plot_importance
from keras.callbacks import EarlyStopping, ModelCheckpoint

C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
# load model
path = os.getcwd()
model = pickle.load(open(path + '/Model/1025model_xgb.h5', 'rb'))

## Prepare Functions 

In [5]:
# Functions for bounding rectangle
def calc_bounding_rect(image, landmarks): # Calculate bounding rectangle 
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]
        landmark_array = np.append(landmark_array, landmark_point, axis=0)
    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


def draw_bounding_rect(use_brect, image, brect): # Draw bounding rectangle
    if use_brect:
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)
    return image

In [6]:
def preprocessed_data(data) :
    # drop unnecessary column: landmark
    data = data.drop([0], axis=1)
        
    # mapping hand position: right, left 
    data[1] = data[1].replace({'Right': 0, 'Left': 1})
    
    # rename columns 
    data.columns = ["hand_type", "1_x", "1_y", "1_z", "1_rx", "1_ry", "1_rz"]
    
    # add new columns 
    new_cols = []
    for i in range(2, 22) :
        new_cols = new_cols + [str(i)+"_x", str(i)+"_y", str(i)+"_z", str(i)+"_rx", str(i)+"_ry", str(i)+"_rz"]
    data[new_cols] = ""
    #data = pd.concat([data, data_new])
    
    # insert values at new columns 
    for j in range(1, 21) :
        col_list = []
        col_list = [str(j+1)+"_x", str(j+1)+"_y", str(j+1)+"_z", str(j+1)+"_rx", str(j+1)+"_ry", str(j+1)+"_rz"]
        data.loc[0, col_list] = data.loc[j, ["1_x", "1_y", "1_z", "1_rx", "1_ry", "1_rz"]].values
            
    # remove all rows except for the first row  
    remove_idx = []
    for i in range (1, 21) : 
        remove_idx.append(i)
    data = data.drop(data.index[remove_idx]).reset_index(drop=True)

    # convert dtypes 
    data = data.astype(float)
    
    return data  

## Start Camera with ML
To test demo code with Machine Learning

- to quit the code, press 'q'

In [4]:
# Setting for Mediapipe 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1,              # only two hand detected 
                       min_detection_confidence=0.7) # defualt 0.5

In [ ]:
# Initialize for datasets 
img_cnt = 1
landmarks = []
df = pd.DataFrame()

# Initialize the webcam 
video_capture = cv2.VideoCapture(0)

while True:
    # Read each frame from the webcam
    _, frame = video_capture.read()
    x, y, c = frame.shape
    
    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(framergb)
    if result.multi_hand_landmarks:
        for handslms, handness in zip(result.multi_hand_landmarks, 
                                      result.multi_handedness): 
            landmarks.clear() # for empty list 
            for point in mp_hands.HandLandmark: # 0 ~ 20
                x = handslms.landmark[point].x
                y = handslms.landmark[point].y
                z = handslms.landmark[point].z
                landmarks.append([str(point), handness.classification[0].label, x, y, z])
            
            # Drawing landmarks on frames with bounding rectangle
            #brect = calc_bounding_rect(frame, handslms)
            #frame = draw_bounding_rect(True, frame, brect)
            mp_drawing.draw_landmarks(frame, handslms, mp_hands.HAND_CONNECTIONS)
            
    # create realtive coordinates  
    df = pd.DataFrame(landmarks)
    relative_df = pd.DataFrame()
    base_lm = df.iloc[0, 2:].astype(float)
    for i in range(0, 21) :
        target_lm = df.loc[i, 2:].values.astype(float)
        result_lm = target_lm-base_lm
        relative_df = relative_df.append(pd.DataFrame(result_lm).transpose(), ignore_index=True)
    new_df = pd.concat([df, relative_df], axis=1)
    
    # preprocessing for input dataset 
    processed_df = preprocessed_data(new_df)
    
    # input data to machine learning
    X = np.array(processed_df)
    #X = X.reshape(1, 300, 300, 1)
    pred_proba = model.predict_proba(X)
    prediction = np.argmax(pred_proba)
    cv2.putText(frame, str(prediction), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
    # To Quit from application, press "q"
    if cv2.waitKey(1) == ord('q'): 
        img_cnt -= 1
        break
        
    # Show the final output
    cv2.imshow("Output", frame)
    
# release the webcam and destroy all active windows
print("{} images captured!".format(img_cnt))
video_capture.release()
cv2.destroyAllWindows()

In [8]:
# Check one more time, perfect
pred_proba = model.predict_proba(X)
prediction = np.argmax(pred_proba)
prediction

7